# Separate data into different positions and do feature selection in order to train different models

In [2]:
import os
from src.features.data_engineering import get_merged_seasons_data

In [3]:
data = get_merged_seasons_data()

## Separate data into different positions

In [4]:
data_gk = data[data['position'] == 'GK']
data_def = data[data['position'] == 'DEF']
data_mid = data[data['position'] == 'MID']
data_fwd = data[data['position'] == 'FWD']

In [5]:
print('GK len:', len(data_gk))
print('DEF len:', len(data_def))
print('MID len:', len(data_mid))
print('FWD len:', len(data_fwd))

GK len: 15267
DEF len: 47611
MID len: 57922
FWD len: 19508


## Feature selection for different positions

In [6]:
# preview columns from data
data.columns

Index(['name', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity',
       'element', 'fixture', 'goals_conceded', 'goals_scored', 'ict_index',
       'influence', 'kickoff_time', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves',
       'selected', 'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'GW', 'position', 'season'],
      dtype='object')

Remember about so called 'target features':
- (['name', 'GW', 'element', 'total_points_next_gameweek', 'season'])

And about features dropped during preprocessing:
- (['fixture', 'kickoff_time', 'opponent_team', 'round', 'transfers_balance', 'was_home'])

### Select features to drop

In [7]:
data_gk.describe()

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,...,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,yellow_cards,GW
count,15267.000000,15267.000000,15267.000000,15267.000000,15267.000000,15267.000000,15267.000000,15267.000000,15267.000000,15267.000000,...,15262.000000,15262.000000,15267.000000,15267.000000,1.526700e+04,15267.000000,1.526700e+04,15267.000000,15267.000000,15267.000000
mean,0.000852,0.082203,5.474553,0.084365,0.079708,332.074998,199.043034,0.411803,0.000066,0.675909,...,1.239811,1.502359,0.006681,1.091308,5.298106e+02,7336.096679,6.806286e+03,44.905875,0.011397,20.972228
std,0.029169,0.421938,9.313818,0.277943,0.926182,189.028235,108.228868,0.937661,0.008093,1.301005,...,1.215960,1.305039,0.260923,2.273843,2.503994e+04,22115.549325,1.990492e+04,4.848968,0.106151,11.541131
min,0.000000,0.000000,-7.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-3.000000,-1.035061e+06,0.000000,0.000000e+00,39.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,166.000000,107.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,-1.057000e+03,49.000000,1.390000e+02,40.000000,0.000000,11.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,333.000000,203.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,-7.200000e+01,331.000000,8.720000e+02,44.000000,0.000000,21.000000
75%,0.000000,0.000000,11.000000,0.000000,0.000000,506.000000,293.000000,0.000000,0.000000,0.900000,...,2.000000,2.000000,0.000000,1.000000,7.850000e+01,4437.500000,6.439000e+03,48.000000,0.000000,30.000000
max,1.000000,3.000000,54.000000,1.000000,30.000000,708.000000,380.000000,9.000000,1.000000,10.300000,...,9.000000,9.000000,18.000000,17.000000,4.190200e+05,459020.000000,1.039664e+06,63.000000,1.000000,47.000000


In [8]:
drop_features_gk = ['assists', 'creativity', 'goals_scored', 'ict_index', 'own_goals', 'penalties_missed', 'red_cards', 'threat', 'yellow_cards']

In [9]:
data_def.describe()

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,...,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,yellow_cards,GW
count,47611.000000,47611.000000,47611.000000,47611.000000,47611.000000,47611.000000,47611.000000,47611.000000,47611.000000,47611.000000,...,47586.000000,47586.000000,47611.000000,47611.000000,4.761100e+04,47611.000000,4.761100e+04,47611.000000,47611.000000,47611.000000
mean,0.025540,0.099998,6.959148,0.111088,3.384966,302.160866,197.046964,0.548382,0.016950,1.426072,...,1.241773,1.505506,2.933482,1.290878,6.671422e+02,9150.101783,8.482334e+03,46.704039,0.063746,20.743022
std,0.166821,0.456991,9.874363,0.314244,8.466178,179.302312,109.321642,1.028448,0.131663,2.263910,...,1.219413,1.304715,8.129947,2.504407,3.699204e+04,31749.682901,2.771543e+04,6.213925,0.244302,11.633362
min,0.000000,0.000000,-12.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-7.000000,-1.012731e+06,0.000000,0.000000e+00,37.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,148.000000,103.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,-1.400000e+03,54.000000,1.710000e+02,43.000000,0.000000,11.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,300.000000,201.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,-7.400000e+01,436.000000,1.185000e+03,45.000000,0.000000,21.000000
75%,0.000000,0.000000,13.000000,0.000000,1.300000,446.000000,292.000000,1.000000,0.000000,2.400000,...,2.000000,2.000000,1.000000,1.000000,1.670000e+02,4179.000000,6.036500e+03,50.000000,0.000000,30.000000
max,3.000000,3.000000,75.000000,1.000000,109.500000,730.000000,380.000000,9.000000,2.000000,22.800000,...,9.000000,9.000000,152.000000,24.000000,9.350820e+05,941215.000000,1.025064e+06,85.000000,1.000000,47.000000


In [10]:
drop_features_def = ['own_goals', 'penalties_missed', 'penalties_saved', 'red_cards', 'saves']

In [11]:
data_mid.describe()

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,...,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,yellow_cards,GW
count,57922.000000,57922.000000,57922.000000,57922.000000,57922.000000,57922.000000,57922.000000,57922.000000,57922.000000,57922.000000,...,57900.000000,57900.000000,57922.000000,57922.000000,5.792200e+04,5.792200e+04,5.792200e+04,57922.000000,57922.000000,57922.000000
mean,0.056507,0.093885,5.719157,0.108784,7.119506,308.469528,198.174735,0.488243,0.054090,2.059801,...,1.247237,1.519326,6.587376,1.398104,9.341700e+02,1.053914e+04,9.604768e+03,54.150616,0.059632,20.866994
std,0.253225,0.463168,9.237676,0.311371,13.381443,180.338521,109.328979,0.944814,0.251359,3.466613,...,1.224810,1.317185,14.375928,2.470625,5.435454e+04,4.686353e+04,3.950957e+04,13.225818,0.236806,11.638579
min,0.000000,0.000000,-14.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-2.000000,-1.857821e+06,0.000000e+00,0.000000e+00,40.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,155.000000,104.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,-8.600000e+02,2.700000e+01,6.600000e+01,45.000000,0.000000,11.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,301.000000,202.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,-4.600000e+01,2.050000e+02,5.600000e+02,50.000000,0.000000,21.000000
75%,0.000000,0.000000,9.000000,0.000000,10.700000,458.000000,293.000000,1.000000,0.000000,3.000000,...,2.000000,2.000000,6.000000,2.000000,1.100000e+01,2.345000e+03,4.295000e+03,57.000000,0.000000,30.000000
max,4.000000,3.000000,93.000000,1.000000,170.900000,737.000000,380.000000,9.000000,4.000000,35.800000,...,9.000000,9.000000,181.000000,29.000000,1.907229e+06,1.991731e+06,1.872898e+06,136.000000,1.000000,47.000000


In [12]:
drop_features_mid = ['own_goals', 'penalties_missed', 'penalties_saved', 'red_cards', 'saves']

In [13]:
data_fwd.describe()

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,...,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,yellow_cards,GW
count,19508.000000,19508.000000,19508.000000,19508.000000,19508.000000,19508.000000,19508.000000,19508.000000,19508.000000,19508.000000,...,19501.000000,19501.000000,19508.000000,19508.000000,1.950800e+04,1.950800e+04,1.950800e+04,19508.000000,19508.000000,19508.000000
mean,0.051825,0.161882,4.733084,0.087144,4.573139,320.552286,198.745694,0.432489,0.109596,2.210801,...,1.229937,1.507666,11.464835,1.453660,1.849246e+03,2.020592e+04,1.835548e+04,60.694484,0.039984,20.916752
std,0.242655,0.617526,11.029191,0.282053,9.063182,185.651113,109.087114,0.891628,0.367440,3.959847,...,1.212308,1.290997,21.433914,2.707575,7.775192e+04,6.638475e+04,5.686969e+04,17.119851,0.195926,11.603276
min,0.000000,0.000000,-19.000000,0.000000,0.000000,4.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-2.000000,-1.734284e+06,0.000000e+00,0.000000e+00,42.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,153.000000,105.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,-2.237750e+03,8.000000e+01,2.407500e+02,49.000000,0.000000,11.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,327.000000,203.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,-9.500000e+01,8.375000e+02,1.878500e+03,56.000000,0.000000,21.000000
75%,0.000000,0.000000,3.000000,0.000000,3.800000,480.000000,294.000000,1.000000,0.000000,3.000000,...,2.000000,2.000000,16.000000,2.000000,1.280000e+02,8.450000e+03,1.197575e+04,65.000000,0.000000,30.000000
max,4.000000,3.000000,128.000000,1.000000,92.000000,735.000000,380.000000,9.000000,4.000000,31.500000,...,9.000000,9.000000,199.000000,24.000000,1.983733e+06,2.104464e+06,1.782662e+06,132.000000,1.000000,47.000000


In [14]:
drop_features_fwd = ['clean_sheets', 'goals_conceded', 'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards', 'saves']

### Drop features and save data to intermediate folder

In [15]:
data_gk = data_gk.drop(drop_features_gk, axis=1)
data_def = data_def.drop(drop_features_def, axis=1)
data_mid = data_mid.drop(drop_features_mid, axis=1)
data_fwd = data_fwd.drop(drop_features_fwd, axis=1)

In [16]:
data_intermediate_path = os.path.dirname(os.getcwd()) + '\\data\\intermediate\\'

In [17]:
data_gk.to_csv(data_intermediate_path + 'separate/gk_full.csv', index=False)
data_def.to_csv(data_intermediate_path + 'separate/def_full.csv', index=False)
data_mid.to_csv(data_intermediate_path + 'separate/mid_full.csv', index=False)
data_fwd.to_csv(data_intermediate_path + 'separate/fwd_full.csv', index=False)

## Separate data for GK and Field players

In [18]:
data_field_players = data.drop(data_gk.index)
print('GK len:', len(data_gk))
print('Field players:', len(data_field_players))

GK len: 15267
Field players: 125041


In [19]:
drop_features_field = ['own_goals', 'penalties_missed', 'penalties_saved', 'red_cards', 'saves']
data_field_players = data_field_players.drop(drop_features_field, axis=1)

In [20]:
data_field_players.to_csv(data_intermediate_path + 'separate/field_full.csv', index=False)